In [18]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_z


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException
from enum import Enum
from pprint import pprint

class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0 
)

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)



texts = [
    "이 영화 정말 좋아요! 완전 만족스러워요.",
    "영화가 전개가 너무 느리고 설명이 부족했습니다.",
    "이 영화 그런대로 괜찮습니다.",
    "전개는 빠르지만 개연성이 아쉽습니다.",
    "최고의 작품이었습니다.",
    "이런 영화가 또 있을지 모르겠네요"
]


def safe_sentiment_analysis(text, use_fixing_parser=True):
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."


def run_sentiment_analysis():
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
        else:
            print(f"   오류: {error} ")
            
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
            else:
                print(f"   재시도 실패: {error2} ")
    

run_sentiment_analysis()



1. 텍스트: 이 영화 정말 좋아요! 완전 만족스러워요.
   감정: 긍정 

2. 텍스트: 영화가 전개가 너무 느리고 설명이 부족했습니다.
   감정: 부정 

3. 텍스트: 이 영화 그런대로 괜찮습니다.
   감정: 보통 

4. 텍스트: 전개는 빠르지만 개연성이 아쉽습니다.
   감정: 부정 

5. 텍스트: 최고의 작품이었습니다.
   감정: 긍정 

6. 텍스트: 1000년에 한 번만 나와야하는 영화
   감정: 긍정 
